## Boosting Classification Models 

[Tutorial: CatBoost Overview](https://www.kaggle.com/mitribunskiy/tutorial-catboost-overview/comments) \
[Model Hyperparameter Tuning and Optimization(CatBoost)](https://medium.com/aiplusoau/hyperparameter-tuning-a5fe69d2a6c7)

CatBoost [https://catboost.ai/docs/concepts/python-reference_catboostclassifier.html] \
CatBoost(Cross-Validation) [https://catboost.ai/docs/concepts/python-reference_cv.html] \

Score = 0.01440


In [1]:
!pip install  catboost

In [2]:
import catboost
print(catboost.__version__)

0.26.1


In [3]:
import numpy as np
import pandas as pd

from catboost import CatBoostClassifier, cv, Pool

from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split

## Load and Preprocessing

In [4]:
train = pd.read_csv('data/train.csv' )
test = pd.read_csv('data/test.csv')
train

,season,date,league_id,league,Team 1,Team2,SPI1,SPI2,proj_score1,proj_score2,...,importance2,score1,score2,xg1,xg2,nsxg1,nsxg2,adj_score1,adj_score2,Outcome
0,2019,01/03/19,1979,Chinese Super League,Shandong Luneng,Guizhou Renhe,48.22,37.83,1.75,0.84,...,22.1,1.0,0.0,1.39,0.26,2.05,0.54,1.05,0.00,1
1,2019,01/03/19,1979,Chinese Super League,Shanghai Greenland,Shanghai SIPG,39.81,60.08,1.22,1.89,...,63.4,0.0,4.0,0.57,2.76,0.80,1.50,0.00,3.26,0
2,2019,01/03/19,1979,Chinese Super League,Guangzhou Evergrande,Tianjin Quanujian,65.59,39.99,2.58,0.62,...,28.8,3.0,0.0,0.49,0.45,1.05,0.75,3.15,0.00,1
3,2019,01/03/19,1979,Chinese Super League,Wuhan Zall,Beijing Guoan,32.25,54.82,1.10,1.79,...,58.9,0.0,1.0,1.12,0.97,1.51,0.94,0.00,1.05,0
4,2019,01/03/19,1979,Chinese Super League,Chongqing Lifan,Guangzhou RF,38.24,40.45,1.57,1.24,...,21.3,2.0,2.0,2.77,3.17,1.05,2.08,2.10,2.10,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7438,2021,12/12/21,1846,German 2. Bundesliga,Karlsruher SC,1. FC Heidenheim 1846,34.05,33.76,1.39,1.19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
7439,2021,12/12/21,1846,German 2. Bundesliga,SC Paderborn,SV Darmstadt 98,36.73,36.11,1.67,1.44,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
7440,2021,14/12/21,1983,South African ABSA Premier League,Moroka Swallows,Orlando Pirates,12.97,23.59,1.05,1.50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
7441,2021,14/12/21,1983,South African ABSA Premier League,Tshakhuma Tsha Madzivhandila,Black Aces,11.76,18.07,1.22,1.46,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [5]:
# train
train['date_time'] = pd.to_datetime(train['date'], format='%d/%m/%y')
#train['year'] = train['date_time'].dt.year
train['month'] = train['date_time'].dt.month
train['weekday'] = train['date_time'].dt.weekday

In [6]:
# test
test['date_time'] = pd.to_datetime(test['date'], format='%d/%m/%y')
#test['year'] = test['date_time'].dt.year
test['month'] = test['date_time'].dt.month
test['weekday'] = test['date_time'].dt.weekday

In [7]:
train.columns

Index(['season', 'date', 'league_id', 'league', 'Team 1', 'Team2', 'SPI1',
       'SPI2', 'proj_score1', 'proj_score2', 'importance1', 'importance2',
       'score1', 'score2', 'xg1', 'xg2', 'nsxg1', 'nsxg2', 'adj_score1',
       'adj_score2', 'Outcome', 'date_time', 'month', 'weekday'],
      dtype='object')

In [8]:
features = ['season',
                'league_id', 'Team 1', 'Team2',
                'SPI1', 'SPI2', 'proj_score1', 'proj_score2',
            
                # with nan:
                'importance1', 'importance2',
                'score1', 'score2',
                'xg1', 'xg2', 
                'nsxg1', 'nsxg2', 
                'adj_score1', 'adj_score2',
                
                # from date:
                #'month', 
                #'weekday'
               ]

cat_features = [#'season',
                'league_id', 'Team 1', 'Team2',
    
                #'month', 
                #'weekday'
               ]


In [9]:
X = train[features]
y = train['Outcome']

In [10]:
SEED = 42

In [11]:
# Let's split the train data into training and validation sets.

X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, random_state=SEED)

In [12]:
# Test data
X_test = test[features]

In [13]:
# Create Pool data
train_data = Pool(data=X_train,
                  label=y_train,
                  cat_features=cat_features
                 )

valid_data = Pool(data=X_valid,
                  label=y_valid,
                  cat_features=cat_features
                 )

### Model Training

In [107]:
CATBOOST_PARAMS = {
    'random_seed':SEED,
    'learning_rate':0.01,
#    'auto_class_weights':'Balanced',
    'iterations':2000,
    'loss_function':'Logloss',
#    'eval_metric':'AUC',   
    'max_depth':8,
    'early_stopping_rounds':20,
    'min_data_in_leaf':5}

In [108]:
%%time
model = CatBoostClassifier(**CATBOOST_PARAMS)

model.fit(train_data,
          eval_set=(valid_data),
          use_best_model=True, 
         logging_level= 'Silent',  #'Verbose',  # you can uncomment this for text output
         plot=True
        )

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

CPU times: user 1min 3s, sys: 2.51 s, total: 1min 5s
Wall time: 13.4 s


In [109]:
# Evaluate with predict
log_loss(y_valid, model.predict(X_valid))

0.30155116878163307

In [110]:
# Evaluate with predict_proba !!!
log_loss(y_valid, model.predict_proba(X_valid))

0.03178564815820114

### Predict and Submit

In [111]:
# with Labels
submission = pd.read_csv('data/submission.csv' )
submission['Outcome'] = model.predict(X_test)
submission.to_csv('cat_submission_new.csv',index=False, header=True)
submission

,Outcome
0,1
1,1
2,0
3,1
4,0
...,...
4003,1
4004,0
4005,1
4006,1


In [112]:
submission['Outcome'].value_counts()

1    2704
0    1304
Name: Outcome, dtype: int64

In [113]:
# with Probabilities
preds = model.predict_proba(X_test)
submission['Outcome'] = [pred[1] for pred in preds]
#submission.to_csv('cat_proba_submission.csv',index=False, header=True)
submission

,Outcome
0,0.999993
1,0.999953
2,0.000024
3,0.945629
4,0.000025
...,...
4003,0.714726
4004,0.259715
4005,0.999079
4006,0.999971


###  Model Cross-Validation and randomized_search

In [114]:
all_train_data = Pool(data=X,
                      label=y,
                      cat_features=cat_features
                     )

In [115]:
%%time
grid = {'learning_rate': [0.3, 0.5, 0.4, 0.1,0.05,0.01],
        'max_depth': [4,  6,  8,  10],
        'l2_leaf_reg': [1,  3,  5,  7, 9],
        #'grow_policy': ['SymmetricTree', 'Depthwise', 'Lossguide']
       }

model = CatBoostClassifier(iterations=2000)


randomized_search_result = model.randomized_search(grid,
            X=all_train_data,
            cv=5,
            partition_random_seed=SEED,
            search_by_train_test_split=True,
            refit=True,
            shuffle=True,
            stratified=None,
            train_size=0.8,
            verbose=0,
            plot=False)

0:	learn: 0.4891454	test: 0.4961435	best: 0.4961435 (0)	total: 5.59ms	remaining: 11.2s
1:	learn: 0.3655474	test: 0.3759715	best: 0.3759715 (1)	total: 8.06ms	remaining: 8.05s
2:	learn: 0.2985725	test: 0.3092871	best: 0.3092871 (2)	total: 10.3ms	remaining: 6.88s
3:	learn: 0.2416139	test: 0.2510088	best: 0.2510088 (3)	total: 12.5ms	remaining: 6.26s
4:	learn: 0.1983736	test: 0.2072028	best: 0.2072028 (4)	total: 14.6ms	remaining: 5.84s
5:	learn: 0.1769207	test: 0.1839742	best: 0.1839742 (5)	total: 16.9ms	remaining: 5.61s
6:	learn: 0.1586002	test: 0.1654079	best: 0.1654079 (6)	total: 19.1ms	remaining: 5.43s
7:	learn: 0.1452217	test: 0.1510011	best: 0.1510011 (7)	total: 21.8ms	remaining: 5.42s
8:	learn: 0.1322992	test: 0.1368124	best: 0.1368124 (8)	total: 24ms	remaining: 5.31s
9:	learn: 0.1222950	test: 0.1259830	best: 0.1259830 (9)	total: 26.8ms	remaining: 5.33s
10:	learn: 0.1147372	test: 0.1182960	best: 0.1182960 (10)	total: 29.1ms	remaining: 5.27s
11:	learn: 0.1089286	test: 0.1115638	best: 

174:	learn: 0.0343692	test: 0.0363785	best: 0.0363626 (171)	total: 402ms	remaining: 4.19s
175:	learn: 0.0343672	test: 0.0363760	best: 0.0363626 (171)	total: 405ms	remaining: 4.2s
176:	learn: 0.0343667	test: 0.0363758	best: 0.0363626 (171)	total: 407ms	remaining: 4.2s
177:	learn: 0.0343623	test: 0.0363763	best: 0.0363626 (171)	total: 410ms	remaining: 4.2s
178:	learn: 0.0343623	test: 0.0363764	best: 0.0363626 (171)	total: 412ms	remaining: 4.2s
179:	learn: 0.0343622	test: 0.0363764	best: 0.0363626 (171)	total: 414ms	remaining: 4.19s
180:	learn: 0.0343611	test: 0.0363761	best: 0.0363626 (171)	total: 416ms	remaining: 4.18s
181:	learn: 0.0343464	test: 0.0363469	best: 0.0363469 (181)	total: 428ms	remaining: 4.27s
182:	learn: 0.0342857	test: 0.0362459	best: 0.0362459 (182)	total: 430ms	remaining: 4.27s
183:	learn: 0.0342855	test: 0.0362460	best: 0.0362459 (182)	total: 432ms	remaining: 4.26s
184:	learn: 0.0339226	test: 0.0357885	best: 0.0357885 (184)	total: 435ms	remaining: 4.27s
185:	learn: 0.

338:	learn: 0.0247367	test: 0.0296701	best: 0.0296536 (319)	total: 798ms	remaining: 3.91s
339:	learn: 0.0245875	test: 0.0294959	best: 0.0294959 (339)	total: 801ms	remaining: 3.91s
340:	learn: 0.0245875	test: 0.0294959	best: 0.0294959 (339)	total: 803ms	remaining: 3.91s
341:	learn: 0.0245874	test: 0.0294961	best: 0.0294959 (339)	total: 806ms	remaining: 3.91s
342:	learn: 0.0245526	test: 0.0294941	best: 0.0294941 (342)	total: 808ms	remaining: 3.9s
343:	learn: 0.0245277	test: 0.0294970	best: 0.0294941 (342)	total: 810ms	remaining: 3.9s
344:	learn: 0.0242897	test: 0.0292739	best: 0.0292739 (344)	total: 813ms	remaining: 3.9s
345:	learn: 0.0242896	test: 0.0292740	best: 0.0292739 (344)	total: 815ms	remaining: 3.89s
346:	learn: 0.0242888	test: 0.0292739	best: 0.0292739 (344)	total: 817ms	remaining: 3.89s
347:	learn: 0.0241444	test: 0.0292362	best: 0.0292362 (347)	total: 819ms	remaining: 3.89s
348:	learn: 0.0241444	test: 0.0292362	best: 0.0292362 (347)	total: 821ms	remaining: 3.88s
349:	learn: 0

431:	learn: 0.0222361	test: 0.0285541	best: 0.0285541 (431)	total: 995ms	remaining: 3.61s
432:	learn: 0.0222009	test: 0.0285642	best: 0.0285541 (431)	total: 998ms	remaining: 3.61s
433:	learn: 0.0220743	test: 0.0283553	best: 0.0283553 (433)	total: 1s	remaining: 3.61s
434:	learn: 0.0220222	test: 0.0284222	best: 0.0283553 (433)	total: 1s	remaining: 3.61s
435:	learn: 0.0220221	test: 0.0284220	best: 0.0283553 (433)	total: 1.01s	remaining: 3.61s
436:	learn: 0.0220220	test: 0.0284221	best: 0.0283553 (433)	total: 1.01s	remaining: 3.62s
437:	learn: 0.0220197	test: 0.0284236	best: 0.0283553 (433)	total: 1.01s	remaining: 3.61s
438:	learn: 0.0219645	test: 0.0284637	best: 0.0283553 (433)	total: 1.01s	remaining: 3.61s
439:	learn: 0.0219623	test: 0.0284640	best: 0.0283553 (433)	total: 1.02s	remaining: 3.61s
440:	learn: 0.0219448	test: 0.0284594	best: 0.0283553 (433)	total: 1.02s	remaining: 3.6s
441:	learn: 0.0218969	test: 0.0284678	best: 0.0283553 (433)	total: 1.02s	remaining: 3.6s
442:	learn: 0.0218

530:	learn: 0.0195156	test: 0.0274107	best: 0.0273817 (511)	total: 1.2s	remaining: 3.31s
531:	learn: 0.0195156	test: 0.0274107	best: 0.0273817 (511)	total: 1.2s	remaining: 3.31s
532:	learn: 0.0195156	test: 0.0274107	best: 0.0273817 (511)	total: 1.2s	remaining: 3.31s
533:	learn: 0.0195156	test: 0.0274109	best: 0.0273817 (511)	total: 1.21s	remaining: 3.31s
534:	learn: 0.0194953	test: 0.0274129	best: 0.0273817 (511)	total: 1.21s	remaining: 3.31s
535:	learn: 0.0193143	test: 0.0271263	best: 0.0271263 (535)	total: 1.21s	remaining: 3.3s
536:	learn: 0.0193143	test: 0.0271263	best: 0.0271263 (536)	total: 1.21s	remaining: 3.3s
537:	learn: 0.0192904	test: 0.0271470	best: 0.0271263 (536)	total: 1.21s	remaining: 3.3s
538:	learn: 0.0192904	test: 0.0271470	best: 0.0271263 (536)	total: 1.21s	remaining: 3.29s
539:	learn: 0.0192904	test: 0.0271470	best: 0.0271263 (536)	total: 1.22s	remaining: 3.29s
540:	learn: 0.0192903	test: 0.0271470	best: 0.0271263 (536)	total: 1.22s	remaining: 3.29s
541:	learn: 0.01

...

1294:	learn: 0.0026508	test: 0.0342593	best: 0.0340666 (90)
1295:	learn: 0.0026508	test: 0.0342596	best: 0.0340666 (90)
1296:	learn: 0.0026507	test: 0.0342605	best: 0.0340666 (90)
1297:	learn: 0.0026505	test: 0.0342605	best: 0.0340666 (90)
1298:	learn: 0.0026504	test: 0.0342607	best: 0.0340666 (90)
1299:	learn: 0.0026504	test: 0.0342610	best: 0.0340666 (90)
1300:	learn: 0.0026504	test: 0.0342610	best: 0.0340666 (90)
1301:	learn: 0.0026504	test: 0.0342616	best: 0.0340666 (90)
1302:	learn: 0.0026503	test: 0.0342616	best: 0.0340666 (90)
1303:	learn: 0.0026503	test: 0.0342616	best: 0.0340666 (90)
1304:	learn: 0.0026503	test: 0.0342617	best: 0.0340666 (90)
1305:	learn: 0.0026502	test: 0.0342624	best: 0.0340666 (90)
1306:	learn: 0.0026501	test: 0.0342626	best: 0.0340666 (90)
1307:	learn: 0.0026501	test: 0.0342625	best: 0.0340666 (90)
1308:	learn: 0.0026501	test: 0.0342624	best: 0.0340666 (90)
1309:	learn: 0.0026501	test: 0.0342628	best: 0.0340666 (90)
1310:	learn: 0.0026501	test: 0.0342627	b

1435:	learn: 0.0025877	test: 0.0340775	best: 0.0340666 (90)
1436:	learn: 0.0025876	test: 0.0340767	best: 0.0340666 (90)
1437:	learn: 0.0025876	test: 0.0340767	best: 0.0340666 (90)
1438:	learn: 0.0025876	test: 0.0340767	best: 0.0340666 (90)
1439:	learn: 0.0025875	test: 0.0340771	best: 0.0340666 (90)
1440:	learn: 0.0025875	test: 0.0340773	best: 0.0340666 (90)
1441:	learn: 0.0025875	test: 0.0340773	best: 0.0340666 (90)
1442:	learn: 0.0025874	test: 0.0340774	best: 0.0340666 (90)
1443:	learn: 0.0025874	test: 0.0340776	best: 0.0340666 (90)
1444:	learn: 0.0025873	test: 0.0340785	best: 0.0340666 (90)
1445:	learn: 0.0025873	test: 0.0340788	best: 0.0340666 (90)
1446:	learn: 0.0025873	test: 0.0340789	best: 0.0340666 (90)
1447:	learn: 0.0025872	test: 0.0340790	best: 0.0340666 (90)
1448:	learn: 0.0025872	test: 0.0340789	best: 0.0340666 (90)
1449:	learn: 0.0025871	test: 0.0340791	best: 0.0340666 (90)
1450:	learn: 0.0025871	test: 0.0340798	best: 0.0340666 (90)
1451:	learn: 0.0025871	test: 0.0340798	b

1587:	learn: 0.0025490	test: 0.0341610	best: 0.0340666 (90)
1588:	learn: 0.0025490	test: 0.0341610	best: 0.0340666 (90)
1589:	learn: 0.0025490	test: 0.0341611	best: 0.0340666 (90)
1590:	learn: 0.0025489	test: 0.0341612	best: 0.0340666 (90)
1591:	learn: 0.0025489	test: 0.0341612	best: 0.0340666 (90)
1592:	learn: 0.0025489	test: 0.0341614	best: 0.0340666 (90)
1593:	learn: 0.0025488	test: 0.0341614	best: 0.0340666 (90)
1594:	learn: 0.0025488	test: 0.0341614	best: 0.0340666 (90)
1595:	learn: 0.0025487	test: 0.0341616	best: 0.0340666 (90)
1596:	learn: 0.0025487	test: 0.0341619	best: 0.0340666 (90)
1597:	learn: 0.0025486	test: 0.0341629	best: 0.0340666 (90)
1598:	learn: 0.0025486	test: 0.0341630	best: 0.0340666 (90)
1599:	learn: 0.0025485	test: 0.0341631	best: 0.0340666 (90)
1600:	learn: 0.0025485	test: 0.0341632	best: 0.0340666 (90)
1601:	learn: 0.0025484	test: 0.0341644	best: 0.0340666 (90)
1602:	learn: 0.0025483	test: 0.0341653	best: 0.0340666 (90)
1603:	learn: 0.0025483	test: 0.0341656	b

1722:	learn: 0.0025343	test: 0.0341441	best: 0.0340666 (90)
1723:	learn: 0.0025343	test: 0.0341444	best: 0.0340666 (90)
1724:	learn: 0.0025343	test: 0.0341447	best: 0.0340666 (90)
1725:	learn: 0.0025342	test: 0.0341450	best: 0.0340666 (90)
1726:	learn: 0.0025342	test: 0.0341452	best: 0.0340666 (90)
1727:	learn: 0.0025341	test: 0.0341455	best: 0.0340666 (90)
1728:	learn: 0.0025341	test: 0.0341457	best: 0.0340666 (90)
1729:	learn: 0.0025340	test: 0.0341458	best: 0.0340666 (90)
1730:	learn: 0.0025340	test: 0.0341457	best: 0.0340666 (90)
1731:	learn: 0.0025340	test: 0.0341460	best: 0.0340666 (90)
1732:	learn: 0.0025340	test: 0.0341461	best: 0.0340666 (90)
1733:	learn: 0.0025339	test: 0.0341461	best: 0.0340666 (90)
1734:	learn: 0.0025338	test: 0.0341460	best: 0.0340666 (90)
1735:	learn: 0.0025338	test: 0.0341462	best: 0.0340666 (90)
1736:	learn: 0.0025337	test: 0.0341462	best: 0.0340666 (90)
1737:	learn: 0.0025337	test: 0.0341465	best: 0.0340666 (90)
1738:	learn: 0.0025337	test: 0.0341467	b

1882:	learn: 0.0025020	test: 0.0343075	best: 0.0340666 (90)
1883:	learn: 0.0025019	test: 0.0343074	best: 0.0340666 (90)
1884:	learn: 0.0025019	test: 0.0343076	best: 0.0340666 (90)
1885:	learn: 0.0025019	test: 0.0343076	best: 0.0340666 (90)
1886:	learn: 0.0025019	test: 0.0343075	best: 0.0340666 (90)
1887:	learn: 0.0025019	test: 0.0343075	best: 0.0340666 (90)
1888:	learn: 0.0025018	test: 0.0343077	best: 0.0340666 (90)
1889:	learn: 0.0025018	test: 0.0343076	best: 0.0340666 (90)
1890:	learn: 0.0025018	test: 0.0343080	best: 0.0340666 (90)
1891:	learn: 0.0025018	test: 0.0343080	best: 0.0340666 (90)
1892:	learn: 0.0025018	test: 0.0343080	best: 0.0340666 (90)
1893:	learn: 0.0025017	test: 0.0343086	best: 0.0340666 (90)
1894:	learn: 0.0025017	test: 0.0343084	best: 0.0340666 (90)
1895:	learn: 0.0025016	test: 0.0343081	best: 0.0340666 (90)
1896:	learn: 0.0025015	test: 0.0343082	best: 0.0340666 (90)
1897:	learn: 0.0025015	test: 0.0343086	best: 0.0340666 (90)
1898:	learn: 0.0025015	test: 0.0343085	b

In [121]:
# Evaluate   #  the best 0.003807786066605326  score = 0.01440 (175 42)
             
log_loss(y_valid, model.predict(X_valid))

0.04639298319992159

In [120]:
# Predict and submit
# with Probabilities
submission = pd.read_csv('data/submission.csv' )

preds = model.predict_proba(X_test)
#submission['Outcome'] = [pred[1] for pred in preds]
submission['Outcome'] = preds

submission.to_csv('test_full1.csv',index=False, header=True)
submission

,Outcome
0,3.458961e-08
1,9.773796e-06
2,1.000000e+00
3,1.642584e-02
4,9.999997e-01
...,...
4003,7.214930e-02
4004,9.758508e-01
4005,4.449853e-04
4006,5.744166e-06
